In [ ]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 

In [2]:
clmns = [
    "Open 12-11","Open 11-10","Open 10-9","Open 9-8","Open 8-7","Open 7-6","Open 6-5","Open 5-4","Open 4-3","Open 3-2","Open 2-1",
    "Close 12-11","Close 11-10","Close 10-9","Close 9-8","Close 8-7","Close 7-6","Close 6-5","Close 5-4","Close 4-3","Close 3-2","Close 2-1",
    "High 12-11","High 11-10","High 10-9","High 9-8","High 8-7","High 7-6","High 6-5","High 5-4","High 4-3","High 3-2","High 2-1",
    "Low 12-11","Low 11-10","Low 10-9","Low 9-8","Low 8-7","Low 7-6","Low 6-5","Low 5-4","Low 4-3","Low 3-2","Low 2-1",
    "AdjV 12-11","AdjV 11-10","AdjV 10-9","AdjV 9-8","AdjV 8-7","AdjV 7-6","AdjV 6-5","AdjV 5-4","AdjV 4-3","AdjV 3-2","AdjV 2-1",
    "Volume 12-11","Volume 11-10","Volume 10-9","Volume 9-8","Volume 8-7","Volume 7-6","Volume 6-5","Volume 5-4","Volume 4-3","Volume 3-2","Volume 2-1",
    "suggestion"
]

def read_syms_from_txt():
  with open("syms.txt", "r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols


def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols


def spliting(data):
  X = data.drop(["suggestion"], axis=1)
  y = data["suggestion"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest


def scaler(row):
    scaler = MinMaxScaler(feature_range=(-10, 10))
    row = scaler.fit_transform(row)
    return row


def process(data):
    data = data.dropna()
    row = []
    data = np.array(data)
    llst = [0, 1, 2, 3, 4, 5]

    for i in range(12, data.shape[0]):
        grow = []
        srow = []

        for j in llst:

           srow.append([
               data[i-1][j] - data[i-2][j],
               data[i-2][j] - data[i-3][j],
               data[i-3][j] - data[i-4][j],
               data[i-4][j] - data[i-5][j],
               data[i-5][j] - data[i-6][j],
               data[i-6][j] - data[i-7][j],
               data[i-7][j] - data[i-8][j],
               data[i-8][j] - data[i-9][j],
               data[i-9][j] - data[i-10][j],
               data[i-10][j] - data[i-11][j],
               data[i-11][j] - data[i-12][j]
           ])

        for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        sugg = 0
        if data[i][3] > data[i][0]:
            sugg = 1

        arr = np.array(grow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)

In [ ]:
def get_data(symbols):
    unattached_dfs = []
    for symbol in symbols:
         data = yf.download(symbol, period="MAX",
                            interval="1h", progress=False)
         if data.empty:
              pass
         else:
               if data.shape[0] > 12:
                   unattached_dfs.append(
                       process(data))

    data = np.array(unattached_dfs[0])
    for i in unattached_dfs[1:]:
        print(i.shape)
        data = np.append(data, np.array(i), axis=0)
        
    symbol = []

    data = pd.DataFrame(data, columns=clmns)
    data = data.astype(float)
    data = data.astype({'suggestion': 'int'})

    # xTrain, xTest, yTrain, yTest = spliting(data)
    print(data.shape)
    data.to_csv(f"{random.randint(1,10000)}.csv")
    # return xTrain, xTest, yTrain, yTest,data


# symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
# symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]

# symbols = read_syms_from_txt()
symbols = get_crypto_syms()

#pieces = 15
#new_arrays = np.array_split(symbols, pieces)
#for symbols in new_arrays:
get_data(symbols)

# xTrain, xTest, yTrain, yTest,data= get_data(symbols)
# data

(831922, 67)
(671885, 67)

1 Failed download:
- DAI-AUD: 1h data not available for startTime=1587540317 and endTime=1650612317. The requested range must be within the last 730 days.
(747124, 67)
(1156560, 67)
(1305443, 67)
(1119034, 67)
(1238953, 67)


In [ ]:
model = Sequential()



model.add(Dense(3333, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(2222, activation='relu'))
model.add(Dense(2222, activation='relu'))
model.add(Dense(1555, activation='relu'))
model.add(Dense(1111, activation='relu'))
model.add(Dense(1111, activation='relu'))
model.add(Dense(777, activation='relu'))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="adam", loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3333)              186648    
                                                                 
 dense_1 (Dense)             (None, 2222)              7408148   
                                                                 
 dense_2 (Dense)             (None, 2222)              4939506   
                                                                 
 dense_3 (Dense)             (None, 1555)              3456765   
                                                                 
 dense_4 (Dense)             (None, 1111)              1728716   
                                                                 
 dense_5 (Dense)             (None, 1111)              1235432   
                                                                 
 dense_6 (Dense)             (None, 777)               8

In [ ]:
model.fit(xTrain,yTrain,epochs=50,batch_size=10000,validation_data=(xTest,yTest))

Epoch 1/50


In [ ]:
model.save("fffirst_model.h5")
model.fit(xTrain,yTrain,epochs=10,batch_size=256,validation_data=(xTest,yTest))
model.save("sssecond_model.h5")

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive
